In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.storagelevel import StorageLevel

In [2]:
 list= [["""{"id": "1234", "location": {"znd","lmk"},"contact": {"phone": [{"number":"12345","code":"111","altno":"No"}]}, {"phone": [{"number":"55656","code":"222","altno":"Yes"}]}}"""]]
df=spark.createDataFrame(list,['json_string'])
df.show()
display(df)

json_string
"{""id"": ""1234"", ""location"": {""znd"",""lmk""},""contact"": {""phone"": [{""number"":""12345"",""code"":""111"",""altno"":""No""}]}, {""phone"": [{""number"":""55656"",""code"":""222"",""altno"":""Yes""}]}}"


In [3]:
 list= [["""{"id": "1234"}"""]]
df=spark.createDataFrame(list,['json_string'])
df.show()
display(df)

json_string
"{""id"": ""1234""}"


In [4]:
cschema=StructType([StructField("id",MapType(StringType(),StringType()))])

                    

df.withColumn("number", F.get_json_object("json_string","$.id")).drop("json_string").show(truncate=False)

+------+
number|
+------+
null |
+------+

In [5]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.storagelevel import StorageLevel

In [6]:
list=[[ 1  , 0.2  , 0.4 , 0.6],
      [ 2  , 0.7  , 0.9 , 0.5]] 
  
df=spark.createDataFrame(list, ['id','sport','travel','work' ])
df.show()

+---+-----+------+----+
 id|sport|travel|work|
+---+-----+------+----+
 1| 0.2| 0.4| 0.6|
 2| 0.7| 0.9| 0.5|
+---+-----+------+----+

In [7]:
df.withColumn("score", F.explode((F.array(*(x for x in df.columns if x!="id"))))).select("id","score").show()








df.withColumn("score", F.explode(F.array(*(x for x in df.columns if x!="id")))).select("id","score").show()

+---+-----+
 id|score|
+---+-----+
 1| 0.2|
 1| 0.4|
 1| 0.6|
 2| 0.7|
 2| 0.9|
 2| 0.5|
+---+-----+

+---+-----+
 id|score|
+---+-----+
 1| 0.2|
 1| 0.4|
 1| 0.6|
 2| 0.7|
 2| 0.9|
 2| 0.5|
+---+-----+

In [8]:
+------+--------+--------+--------+
| c_id | score  | col_x  | col_y  | 
+------+--------+--------+--------+
| 1    | 0.2    | "abc"  | "abc"  |
+------+--------+--------+--------+
| 1    | 0.4    | "abc"  | "abc"  |
+------+--------+--------+--------+
| 1    | 0.6    | "abc"  | "abc"  |
+------+--------+--------+--------+
| 2    | 0.7    | "abc"  | "abc"  |
+------+--------+--------+--------+
| 2    | 0.9    | "abc"  | "abc"  |
+------+--------+--------+--------+
| 2    | 0.5    | "abc"  | "abc"  |
+------+--------+--------+--------+

In [9]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.storagelevel import StorageLevel

list=[[ 'a',199 , 99,  None],
      [ 'a',145 , 450,  None],
      ['a',195 , 350,  None],
      ['a' ,200,100  ,50],
      ['a' ,150,88  ,30],
      ['b' ,100, 110  ,4],
      ['b' ,95,  95  ,None],
      ['b' ,300,  400  ,9],
      ['b' ,93,  92  ,None]] 

df=spark.createDataFrame(list,['Group','X','Y','Res'])

df.show()

+-----+---+---+----+
Group| X| Y| Res|
+-----+---+---+----+
 a|199| 99|null|
 a|145|450|null|
 a|195|350|null|
 a|200|100| 50|
 a|150| 88| 30|
 b|100|110| 4|
 b| 95| 95|null|
 b|300|400| 9|
 b| 93| 92|null|
+-----+---+---+----+

In [10]:
from pyspark.sql.functions import when
w=Window().partitionBy("Group").orderBy(F.col("Res").asc_nulls_first())
w2=Window().partitionBy("Group").orderBy(F.col("distance").desc())
df.withColumn("nullxy", F.collect_list(F.when(F.col("Res").isNull(),F.array("X","Y")).otherwise(F.array(F.lit(0)))).over(w))\
.withColumn("nullxy", F.array_remove(F.flatten("nullxy"),0))\
.withColumn("xy", F.array("X","Y"))\
.withColumn("distance", F.sqrt(F.pow(F.element_at("nullxy",1)-F.element_at("xy",1),F.lit(2))+\
                                 F.pow(F.element_at("nullxy",2)-F.element_at("xy",2),F.lit(2))))\
.withColumn("lag", F.lag("Res").over(w2))\
.withColumn("Res", F.when(F.col("Res").isNull(), F.col("lag")).otherwise(F.col("Res")))\
.show(truncate=False)

+-----+---+---+----+-----------------------------+----------+------------------+----+
Group|X |Y |Res |nullxy |xy |distance |lag |
+-----+---+---+----+-----------------------------+----------+------------------+----+
b |300|400|9 |[95, 95, 93, 92] |[300, 400]|367.4914965002592 |null|
b |100|110|4 |[95, 95, 93, 92] |[100, 110]|15.811388300841896|9 |
b |93 |92 |4 |[95, 95, 93, 92] |[93, 92] |3.605551275463989 |4 |
b |95 |95 |null|[95, 95, 93, 92] |[95, 95] |0.0 |null|
a |145|450|null|[199, 99, 145, 450, 195, 350]|[145, 450]|355.1295538250794 |null|
a |195|350|null|[199, 99, 145, 450, 195, 350]|[195, 350]|251.03187048659777|null|
a |150|88 |30 |[199, 99, 145, 450, 195, 350]|[150, 88] |50.21951811795888 |null|
a |200|100|50 |[199, 99, 145, 450, 195, 350]|[200, 100]|1.4142135623730951|30 |
a |199|99 |50 |[199, 99, 145, 450, 195, 350]|[199, 99] |0.0 |50 |
+-----+---+---+----+-----------------------------+----------+------------------+----+

In [11]:
from pyspark.sql.functions import when
w=Window().partitionBy("Group").orderBy(F.col("Res").asc_nulls_first())
w2=Window().partitionBy("Group").orderBy(F.col("distance").desc())
df.withColumn("nullxy", F.collect_list(F.when(F.col("Res").isNull(),F.array("X","Y")).otherwise(F.array(F.lit(0)))).over(w))\
.withColumn("nullxy", F.array_remove(F.flatten("nullxy"),0))\
.withColumn("xy", F.array("X","Y"))\
.withColumn("nu")
.show(truncate=False)

+-----+---+---+----+-----------------------------+----------+
Group|X |Y |Res |nullxy |xy |
+-----+---+---+----+-----------------------------+----------+
b |95 |95 |null|[95, 95, 93, 92] |[95, 95] |
b |93 |92 |null|[95, 95, 93, 92] |[93, 92] |
b |100|110|4 |[95, 95, 93, 92] |[100, 110]|
b |300|400|9 |[95, 95, 93, 92] |[300, 400]|
a |199|99 |null|[199, 99, 145, 450, 195, 350]|[199, 99] |
a |145|450|null|[199, 99, 145, 450, 195, 350]|[145, 450]|
a |195|350|null|[199, 99, 145, 450, 195, 350]|[195, 350]|
a |150|88 |30 |[199, 99, 145, 450, 195, 350]|[150, 88] |
a |200|100|50 |[199, 99, 145, 450, 195, 350]|[200, 100]|
+-----+---+---+----+-----------------------------+----------+

In [12]:
w=Window().partitionBy("Group").orderBy(F.col("Res").asc_nulls_first())
w2=Window().partitionBy("Group").orderBy(F.col("distance").desc())
df.withColumn("nullxy", F.collect_list(F.when(F.col("Res").isNull(),F.array("X","Y")).otherwise(F.array(F.lit(0)))).over(w))\
.withColumn("nullxy", F.array_remove(F.flatten("nullxy"),0))\
.withColumn("xy", F.array("X","Y"))\
.withColumn("distance", F.sqrt(F.pow(F.element_at("nullxy",1)-F.element_at("xy",1),F.lit(2))+\
                                 F.pow(F.element_at("nullxy",2)-F.element_at("xy",2),F.lit(2)))).drop("xy","nullxy")\
.withColumn("lag", F.lag("Res").over(w2))\
.withColumn("Res", F.when(F.col("Res").isNull(), F.col("lag")).otherwise(F.col("Res"))).drop("lag","distance")\
.orderBy("Group").show()

+-----+---+---+---+--------+----------+------------------+----+
Group| X| Y|Res| nullxy| xy| distance| lag|
+-----+---+---+---+--------+----------+------------------+----+
 c|324|124| 45|[95, 95]|[324, 124]|230.82894099310857|null|
 c|249| 67| 95|[95, 95]| [249, 67]|156.52475842498527| 45|
 c|129|189| 95|[95, 95]|[129, 189]| 99.9599919967984| 95|
 c| 95| 95| 95|[95, 95]| [95, 95]| 0.0| 95|
 b|100|110| 4| []|[100, 110]| null|null|
 b| 95| 95| 4| []| [95, 95]| null| 4|
 a|150|500| 30| []|[150, 500]| null|null|
 a|200|100| 50| []|[200, 100]| null| 30|
 a|100|100| 50| []|[100, 100]| null| 50|
+-----+---+---+---+--------+----------+------------------+----+

In [14]:
df1.show()

+----------+----------+
 xy| nullxy|
+----------+----------+
 [95, 95]| [95, 95]|
[100, 110]| [95, 95]|
[100, 100]|[100, 100]|
[150, 500]|[100, 100]|
[200, 100]|[100, 100]|
+----------+----------+

In [15]:
def getdistance(a,b):
  import math
  return math.sqrt(((a[0]-b[0])**2) + ((a[1]-b[1])**2))

In [16]:
df1.withColumn("distance", F.sqrt(F.pow(F.element_at("nullxy",1)-F.element_at("xy",1),F.lit(2))+\
                                 F.pow(F.element_at("nullxy",2)-F.element_at("xy",2),F.lit(2)))).show()

+----------+----------+------------------+
 xy| nullxy| distance|
+----------+----------+------------------+
 [95, 95]| [95, 95]| 0.0|
[100, 110]| [95, 95]|15.811388300841896|
[100, 100]|[100, 100]| 0.0|
[150, 500]|[100, 100]|403.11288741492746|
[200, 100]|[100, 100]| 100.0|
+----------+----------+------------------+

In [17]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.storagelevel import StorageLevel

In [18]:
list=[['2020-02-01','2020-03-21']]
df=spark.createDataFrame(list,['start_date','end_date'])
df.show()

+----------+----------+
start_date| end_date|
+----------+----------+
2020-02-01|2020-03-21|
+----------+----------+

In [19]:
display(df.withColumn("start_date",F.to_date("start_date")).withColumn("end_date", F.to_date("end_date"))\
.withColumn("lineofdate", F.expr("""sequence(start_date,end_date,interval 1 day)""")))



start_date,end_date,lineofdate
2020-02-01,2020-03-21,"List(2020-02-01, 2020-02-02, 2020-02-03, 2020-02-04, 2020-02-05, 2020-02-06, 2020-02-07, 2020-02-08, 2020-02-09, 2020-02-10, 2020-02-11, 2020-02-12, 2020-02-13, 2020-02-14, 2020-02-15, 2020-02-16, 2020-02-17, 2020-02-18, 2020-02-19, 2020-02-20, 2020-02-21, 2020-02-22, 2020-02-23, 2020-02-24, 2020-02-25, 2020-02-26, 2020-02-27, 2020-02-28, 2020-02-29, 2020-03-01, 2020-03-02, 2020-03-03, 2020-03-04, 2020-03-05, 2020-03-06, 2020-03-07, 2020-03-08, 2020-03-09, 2020-03-10, 2020-03-11, 2020-03-12, 2020-03-13, 2020-03-14, 2020-03-15, 2020-03-16, 2020-03-17, 2020-03-18, 2020-03-19, 2020-03-20, 2020-03-21)"


In [20]:
list=[['2019-01-01 00:49:00.00']]

df=spark.createDataFrame(list,['datetime'])

df.show(truncate=False)

+----------------------+
datetime |
+----------------------+
2019-01-01 00:49:00.00|
+----------------------+

In [21]:
df.withColumn('datetime_UTC',  F.to_utc_timestamp(F.col('datetime'), 'SystemV/CST6CDT')).show()

+--------------------+-------------------+
 datetime| datetime_UTC|
+--------------------+-------------------+
2019-01-01 00:49:...|2019-01-01 06:49:00|
+--------------------+-------------------+

In [22]:
list=[['VOD','01-02',10,13,0.05],
      ['VOD','01-02',10,14,0.02],
      ['VOD','01-02',10,16,-0.02], 
      ['VOD','01-02',11,13,0.05],
      ['VOD','01-02',12,3,0.02],
      ['VOD','01-02',13,45,-0.02]]
df=spark.createDataFrame(list,['stock','date','hour','minute','return'])
df.show()

+-----+-----+----+------+------+
stock| date|hour|minute|return|
+-----+-----+----+------+------+
 VOD|01-02| 10| 13| 0.05|
 VOD|01-02| 10| 14| 0.02|
 VOD|01-02| 10| 16| -0.02|
 VOD|01-02| 11| 13| 0.05|
 VOD|01-02| 12| 3| 0.02|
 VOD|01-02| 13| 45| -0.02|
+-----+-----+----+------+------+

In [23]:
def autocorr(ret):
  import pandas as pd
  s = pd.Series(ret)
  return float(s.autocorr(lag=1))

auto=F.udf(autocorr, FloatType())

 

In [24]:
df.show()

def autocorr(ret):
  import pandas as pd
  s = pd.Series(ret)
  return float(s.autocorr(lag=1))

auto=F.udf(autocorr, FloatType())

df.groupBy("stock","date").agg(F.collect_list(F.col("return")).alias("return")).withColumn("auto", auto("return")).select("stock","date","auto").show(truncate=False)

+-----+-----+----+------+------+
stock| date|hour|minute|return|
+-----+-----+----+------+------+
 VOD|01-02| 10| 13| 0.05|
 VOD|01-02| 10| 14| 0.02|
 VOD|01-02| 10| 16| -0.02|
 VOD|01-02| 11| 13| 0.05|
 VOD|01-02| 12| 3| 0.02|
 VOD|01-02| 13| 45| -0.02|
+-----+-----+----+------+------+

+-----+-----+-----------+
stock|date |auto |
+-----+-----+-----------+
VOD |01-02|-0.28925422|
+-----+-----+-----------+

In [25]:
import pandas as pd
s = pd.Series([0.05, 0.02, 0.02, 0.05, -0.02, 0.02])
s.autocorr()

Out[65]: 0.1735525336251558

In [26]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.storagelevel import StorageLevel

In [27]:
list=[[1,0],
      [1,0],
      [1,4],
      [1,0],
      [1,0],
      [1,0]]

df=spark.createDataFrame(list, ['id','partition'])
df.show()

+---+---------+
 id|partition|
+---+---------+
 1| 0|
 1| 0|
 1| 4|
 1| 0|
 1| 0|
 1| 0|
+---+---------+

In [28]:
df.withColumn("partition", F.sum("partition").over(Window().partitionBy("id"))).show()

+---+---------+
 id|partition|
+---+---------+
 1| 4|
 1| 4|
 1| 4|
 1| 4|
 1| 4|
 1| 4|
+---+---------+

In [29]:
df.withColumn("partition", F.mean("partition").over(Window().partitionBy("id"))).show()

+---+------------------+
 id| partition|
+---+------------------+
 1|0.6666666666666666|
 1|0.6666666666666666|
 1|0.6666666666666666|
 1|0.6666666666666666|
 1|0.6666666666666666|
 1|0.6666666666666666|
+---+------------------+

In [30]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.storagelevel import StorageLevel

In [31]:
list=([1,5,4],
    [1,5,None],
    [1,5,1],
    [1,5,4],
    [2,5,1],
    [2,5,2],
    [2,5,None],
    [2,5,None],
     [2,5,4])
df=spark.createDataFrame(list,['I_id','p_id','xyz'])
df.show()

+----+----+----+
I_id|p_id| xyz|
+----+----+----+
 1| 5| 4|
 1| 5|null|
 1| 5| 1|
 1| 5| 4|
 2| 5| 1|
 2| 5| 2|
 2| 5|null|
 2| 5|null|
 2| 5| 4|
+----+----+----+

In [32]:
df1=df.repartition("I_id","p_id")
w= Window().partitionBy("I_id","p_id").orderBy(F.col("xyz").asc_nulls_first())
w2= Window().partitionBy("I_id","p_id").orderBy(F.col("xyz").asc_nulls_first()).rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
df1.withColumn("xyz1",F.count(F.col("xyz").isNotNull()).over(w))\
.withColumn("xyz2", F.max(F.row_number().over(w)).over(w2))\
.withColumn("xyz3", F.first("xyz1").over(w))\
.withColumn("xyz4", F.when((F.col("xyz2")-F.col("xyz3"))%2!=0, (F.col("xyz2")-F.col("xyz3"))+1).otherwise(F.col("xyz2")-F.col("xyz3")))\
.withColumn("xyz4", (F.col("xyz4")/2).cast("int"))\
.withColumn("xyz6", F.col("xyz4")+F.col("xyz3"))\
.withColumn("xyz5", F.row_number().over(w))\
.withColumn("medianr", F.when(F.col("xyz6")==F.col("xyz5"), F.col("xyz")).otherwise(F.lit(None)))\
.withColumn("medianr2", (F.mean("medianr").over(w2)).cast("int"))\
.withColumn("xyz", F.when(F.col("xyz").isNull(), F.col("medianr2")).otherwise(F.col("xyz")))\
.orderBy("I_id").explain()



== Physical Plan ==
Sort [I_id#3837L ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(I_id#3837L ASC NULLS FIRST, 200), [id=#1472]
 +- *(3) Project [I_id#3837L, p_id#3838L, CASE WHEN isnull(xyz#3839L) THEN cast(medianr2#17070 as bigint) ELSE xyz#3839L END AS xyz#17082L, xyz1#17000L, xyz2#17007, xyz3#17015L, xyz4#17030, xyz6#17038L, xyz5#17048, medianr#17058L, medianr2#17070]
 +- Window [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#17000L, xyz2#17007, xyz3#17015L, xyz4#17030, xyz6#17038L, xyz5#17048, medianr#17058L, cast(avg(medianr#17058L) windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$())) as int) AS medianr2#17070], [I_id#3837L, p_id#3838L], [xyz#3839L ASC NULLS FIRST]
 +- *(2) Project [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#17000L, xyz2#17007, xyz3#17015L, xyz4#17030, xyz6#17038L, xyz5#17048, CASE WHEN (xyz6#17038L = cast(xyz5#17048 as bigint)) THEN xyz#3839L END AS medianr#17058L]
 +- *(2) RunningWindowFunction [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#17000L, xyz2#17007, xyz3#17015L, xyz4#17030, xyz6#17038L, row_number() windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS xyz5#17048], [I_id#3837L, p_id#3838L], [xyz#3839L ASC NULLS FIRST]
 +- *(2) Project [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#17000L, xyz2#17007, xyz3#17015L, xyz4#17030, (cast(xyz4#17030 as bigint) + xyz3#17015L) AS xyz6#17038L]
 +- *(2) Project [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#17000L, xyz2#17007, xyz3#17015L, cast((cast(CASE WHEN NOT (((cast(xyz2#17007 as bigint) - xyz3#17015L) % 2) = 0) THEN ((cast(xyz2#17007 as bigint) - xyz3#17015L) + 1) ELSE (cast(xyz2#17007 as bigint) - xyz3#17015L) END as double) / 2.0) as int) AS xyz4#17030]
 +- Window [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#17000L, max(_w4#17008) windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$())) AS xyz2#17007, first(xyz1#17000L, false) windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RangeFrame, unboundedpreceding$(), currentrow$())) AS xyz3#17015L], [I_id#3837L, p_id#3838L], [xyz#3839L ASC NULLS FIRST]
 +- Window [I_id#3837L, p_id#3838L, xyz#3839L, count(1) windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RangeFrame, unboundedpreceding$(), currentrow$())) AS xyz1#17000L, row_number() windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS _w4#17008], [I_id#3837L, p_id#3838L], [xyz#3839L ASC NULLS FIRST]
 +- Sort [I_id#3837L ASC NULLS FIRST, p_id#3838L ASC NULLS FIRST, xyz#3839L ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(I_id#3837L, p_id#3838L, 200), [id=#1458]
 +- *(1) Scan ExistingRDD[I_id#3837L,p_id#3838L,xyz#3839L]

In [33]:
== Physical Plan ==
Sort [I_id#3837L ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(I_id#3837L ASC NULLS FIRST, 200), [id=#1472]
   +- *(3) Project [I_id#3837L, p_id#3838L, CASE WHEN isnull(xyz#3839L) THEN cast(medianr2#17070 as bigint) ELSE xyz#3839L END AS xyz#17082L, xyz1#17000L, xyz2#17007, xyz3#17015L, xyz4#17030, xyz6#17038L, xyz5#17048, medianr#17058L, medianr2#17070]
      +- Window [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#17000L, xyz2#17007, xyz3#17015L, xyz4#17030, xyz6#17038L, xyz5#17048, medianr#17058L, cast(avg(medianr#17058L) windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$())) as int) AS medianr2#17070], [I_id#3837L, p_id#3838L], [xyz#3839L ASC NULLS FIRST]
         +- *(2) Project [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#17000L, xyz2#17007, xyz3#17015L, xyz4#17030, xyz6#17038L, xyz5#17048, CASE WHEN (xyz6#17038L = cast(xyz5#17048 as bigint)) THEN xyz#3839L END AS medianr#17058L]
            +- *(2) RunningWindowFunction [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#17000L, xyz2#17007, xyz3#17015L, xyz4#17030, xyz6#17038L, row_number() windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS xyz5#17048], [I_id#3837L, p_id#3838L], [xyz#3839L ASC NULLS FIRST]
               +- *(2) Project [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#17000L, xyz2#17007, xyz3#17015L, xyz4#17030, (cast(xyz4#17030 as bigint) + xyz3#17015L) AS xyz6#17038L]
                  +- *(2) Project [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#17000L, xyz2#17007, xyz3#17015L, cast((cast(CASE WHEN NOT (((cast(xyz2#17007 as bigint) - xyz3#17015L) % 2) = 0) THEN ((cast(xyz2#17007 as bigint) - xyz3#17015L) + 1) ELSE (cast(xyz2#17007 as bigint) - xyz3#17015L) END as double) / 2.0) as int) AS xyz4#17030]
                     +- Window [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#17000L, max(_w4#17008) windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$())) AS xyz2#17007, first(xyz1#17000L, false) windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RangeFrame, unboundedpreceding$(), currentrow$())) AS xyz3#17015L], [I_id#3837L, p_id#3838L], [xyz#3839L ASC NULLS FIRST]
                        +- Window [I_id#3837L, p_id#3838L, xyz#3839L, count(1) windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RangeFrame, unboundedpreceding$(), currentrow$())) AS xyz1#17000L, row_number() windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS _w4#17008], [I_id#3837L, p_id#3838L], [xyz#3839L ASC NULLS FIRST]
                           +- Sort [I_id#3837L ASC NULLS FIRST, p_id#3838L ASC NULLS FIRST, xyz#3839L ASC NULLS FIRST], false, 0
                              +- Exchange hashpartitioning(I_id#3837L, p_id#3838L, 200), [id=#1458]
                                 +- *(1) Scan ExistingRDD[I_id#3837L,p_id#3838L,xyz#3839L]

In [34]:
== Physical Plan ==
Sort [I_id#3837L ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(I_id#3837L ASC NULLS FIRST, 200), [id=#1313]
   +- *(3) Project [I_id#3837L, p_id#3838L, CASE WHEN isnull(xyz#3839L) THEN cast(medianr2#16965 as bigint) ELSE xyz#3839L END AS xyz#16977L, xyz1#16895L, xyz2#16902, xyz3#16910L, xyz4#16925, xyz6#16933L, xyz5#16943, medianr#16953L, medianr2#16965]
      +- Window [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#16895L, xyz2#16902, xyz3#16910L, xyz4#16925, xyz6#16933L, xyz5#16943, medianr#16953L, cast(avg(medianr#16953L) windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$())) as int) AS medianr2#16965], [I_id#3837L, p_id#3838L], [xyz#3839L ASC NULLS FIRST]
         +- *(2) Project [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#16895L, xyz2#16902, xyz3#16910L, xyz4#16925, xyz6#16933L, xyz5#16943, CASE WHEN (xyz6#16933L = cast(xyz5#16943 as bigint)) THEN xyz#3839L END AS medianr#16953L]
            +- *(2) RunningWindowFunction [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#16895L, xyz2#16902, xyz3#16910L, xyz4#16925, xyz6#16933L, row_number() windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS xyz5#16943], [I_id#3837L, p_id#3838L], [xyz#3839L ASC NULLS FIRST]
               +- *(2) Project [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#16895L, xyz2#16902, xyz3#16910L, xyz4#16925, (cast(xyz4#16925 as bigint) + xyz3#16910L) AS xyz6#16933L]
                  +- *(2) Project [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#16895L, xyz2#16902, xyz3#16910L, cast((cast(CASE WHEN NOT (((cast(xyz2#16902 as bigint) - xyz3#16910L) % 2) = 0) THEN ((cast(xyz2#16902 as bigint) - xyz3#16910L) + 1) ELSE (cast(xyz2#16902 as bigint) - xyz3#16910L) END as double) / 2.0) as int) AS xyz4#16925]
                     +- Window [I_id#3837L, p_id#3838L, xyz#3839L, xyz1#16895L, max(_w4#16903) windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$())) AS xyz2#16902, first(xyz1#16895L, false) windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RangeFrame, unboundedpreceding$(), currentrow$())) AS xyz3#16910L], [I_id#3837L, p_id#3838L], [xyz#3839L ASC NULLS FIRST]
                        +- Window [I_id#3837L, p_id#3838L, xyz#3839L, count(1) windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RangeFrame, unboundedpreceding$(), currentrow$())) AS xyz1#16895L, row_number() windowspecdefinition(I_id#3837L, p_id#3838L, xyz#3839L ASC NULLS FIRST, specifiedwindowframe(RowFrame, unboundedpreceding$(), currentrow$())) AS _w4#16903], [I_id#3837L, p_id#3838L], [xyz#3839L ASC NULLS FIRST]
                           +- Sort [I_id#3837L ASC NULLS FIRST, p_id#3838L ASC NULLS FIRST, xyz#3839L ASC NULLS FIRST], false, 0
                              +- Exchange hashpartitioning(I_id#3837L, p_id#3838L, 200), [id=#1299]
                                 +- *(1) Scan ExistingRDD[I_id#3837L,p_id#3838L,xyz#3839L]